In [79]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import os
import requests


In [80]:
# Load .env environment variables

import datetime as dt
import pytz

_timeNow = dt.datetime.now(pytz.timezone('US/Eastern'))
_1YearsAgo = _timeNow - dt.timedelta(days=360)
_2YearsAgo = _timeNow - dt.timedelta(days=730)
_3YearsAgo = _timeNow - dt.timedelta(days=1095)
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets/v2/assets?asset_class=crypto"
# Create a connection to the API 
api = tradeapi.REST("PK6JCY12U8K9OYRH7Z6Q", "u4tivaUmWg6VrGYLQnGMZ2PI9RDaWVkMkh9WlMhL", ALPACA_API_BASE_URL, api_version="v2")

In [81]:
# Set the ticker symbol and the number of shares to buy
ticker = "LTCUSD"
number_of_shares = 1

# Make API call
LTC_prices = api.get_crypto_bars(ticker, '1day',
                         start=_2YearsAgo.isoformat(),
                         end=None,
                         limit=None).df

In [82]:
#Filter data by exchange
LTC_prices = LTC_prices[LTC_prices.exchange == 'CBSE']

In [83]:
LTC_prices

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2020-10-28 05:00:00+00:00,CBSE,59.69,59.80,54.71,55.84,209664.380369,13698,56.778483
2020-10-29 05:00:00+00:00,CBSE,54.93,55.81,54.07,54.17,24374.638943,2292,54.929554
2020-10-30 05:00:00+00:00,CBSE,54.11,55.35,53.65,54.33,41746.806490,2698,54.564797
2020-10-31 05:00:00+00:00,CBSE,55.78,56.20,54.80,55.20,18483.047012,1642,55.369051
2020-11-01 05:00:00+00:00,CBSE,55.67,56.85,55.37,55.62,31912.976588,2499,56.137080
...,...,...,...,...,...,...,...,...
2022-10-23 05:00:00+00:00,CBSE,52.20,54.22,51.83,53.49,138406.562853,26994,53.073128
2022-10-24 05:00:00+00:00,CBSE,53.46,53.53,52.11,52.78,160757.399118,29405,52.781106
2022-10-25 05:00:00+00:00,CBSE,52.79,56.84,52.27,56.26,249975.682645,40585,54.981791


In [84]:
# Slice to just the `close` column
LTC_signals_df = LTC_prices.loc[:,["close"]]

In [85]:
# Set the short window and long windows
short_window = 15
long_window = 50

In [86]:
# Generate the short and long moving averages (50 and 100 days, respectively)
LTC_signals_df['SMA_fast'] = LTC_signals_df['close'].rolling(window=short_window).mean()
LTC_signals_df['SMA_slow'] = LTC_signals_df['close'].rolling(window=long_window).mean()

# Prepopulate the `Signal` for trading
LTC_signals_df['Signal'] = 0.0
LTC_signals_df = LTC_signals_df.dropna()
LTC_signals_df

,close,SMA_fast,SMA_slow,Signal
timestamp,,,,
2020-12-16 06:00:00+00:00,93.57,80.876000,72.3172,0.0
2020-12-17 06:00:00+00:00,101.17,81.892667,73.2238,0.0
2020-12-18 06:00:00+00:00,106.96,83.083333,74.2796,0.0
2020-12-19 06:00:00+00:00,120.35,85.801333,75.6000,0.0
2020-12-20 06:00:00+00:00,114.65,87.890000,76.7890,0.0
...,...,...,...,...
2022-10-23 05:00:00+00:00,53.49,51.932000,54.5188,0.0
2022-10-24 05:00:00+00:00,52.78,51.825333,54.3658,0.0
2022-10-25 05:00:00+00:00,56.26,52.128667,54.2890,0.0


In [87]:
# Generate the trading signal 0 or 1,
# where 1 is when short-window (SMA50) is greater than the long (SMA 100)
# and 0 otherwise
LTC_signals_df['Signal'] = np.where(
    LTC_signals_df['SMA_fast']> LTC_signals_df['SMA_slow'], 1.0, 0.0)

In [88]:
LTC_signals_df

,close,SMA_fast,SMA_slow,Signal
timestamp,,,,
2020-12-16 06:00:00+00:00,93.57,80.876000,72.3172,1.0
2020-12-17 06:00:00+00:00,101.17,81.892667,73.2238,1.0
2020-12-18 06:00:00+00:00,106.96,83.083333,74.2796,1.0
2020-12-19 06:00:00+00:00,120.35,85.801333,75.6000,1.0
2020-12-20 06:00:00+00:00,114.65,87.890000,76.7890,1.0
...,...,...,...,...
2022-10-23 05:00:00+00:00,53.49,51.932000,54.5188,0.0
2022-10-24 05:00:00+00:00,52.78,51.825333,54.3658,0.0
2022-10-25 05:00:00+00:00,56.26,52.128667,54.2890,0.0


In [89]:
# Use the pct_change function to generate returns from close prices
LTC_signals_df["Actual Returns"] = LTC_signals_df["close"].pct_change()

In [90]:
##### Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
LTC_signals_df['Entry/Exit'] = LTC_signals_df['Signal'].diff()

# Review the DataFrame
LTC_signals_df.loc["2021-02-02":"2022-06-20",:]

,close,SMA_fast,SMA_slow,Signal,Actual Returns,Entry/Exit
timestamp,,,,,,
2021-02-02 06:00:00+00:00,152.44,137.832000,134.5694,1.0,0.130777,0.0
2021-02-03 06:00:00+00:00,151.33,137.842667,135.9950,1.0,-0.007282,0.0
2021-02-04 06:00:00+00:00,148.29,138.216000,137.0894,1.0,-0.020089,0.0
2021-02-05 06:00:00+00:00,158.76,140.150000,138.2412,1.0,0.070605,0.0
2021-02-06 06:00:00+00:00,151.36,141.056667,139.1292,1.0,-0.046611,0.0
...,...,...,...,...,...,...
2022-06-16 05:00:00+00:00,46.03,55.582667,71.1438,0.0,-0.059076,0.0
2022-06-17 05:00:00+00:00,47.43,54.466667,70.0318,0.0,0.030415,0.0
2022-06-18 05:00:00+00:00,46.01,53.376667,68.9298,0.0,-0.029939,0.0


In [91]:
# Visualize exit position relative to close price
exit = LTC_signals_df[LTC_signals_df['Entry/Exit'] == -1.0]['close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

In [92]:
# Visualize entry position relative to close price
entry = LTC_signals_df[LTC_signals_df['Entry/Exit'] == 1.0]['close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

In [93]:
# Visualize close price for the investment
security_close =LTC_signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

In [94]:
# Visualize moving averages
moving_avgs = LTC_signals_df[['SMA_fast', 'SMA_slow']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

In [95]:
moving_avgs

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [96]:
# Create the overlay plot
entry_exit_plot = entry * exit

# Show the plot with a title
entry_exit_plot.opts(
    title="LTC - SMA_fast, SMA_slow, Entry and Exit Points"
)

:Overlay
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

In [97]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 50

In [98]:
# Buy when the dual moving average crossover Signal equals 1
# Otherwise, `Position` should be zero (sell)
LTC_signals_df['Position'] = share_size * LTC_signals_df['Signal']

In [99]:
# Determine the points in time where a position is bought or sold
LTC_signals_df['Entry/Exit Position'] = LTC_signals_df['Position'].diff()

In [100]:
# Multiply the close price by the number of shares held, or the Position
LTC_signals_df['Portfolio Holdings'] = LTC_signals_df['close'] * LTC_signals_df['Position']

# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
LTC_signals_df['Portfolio Cash'] = initial_capital - (LTC_signals_df['close'] * LTC_signals_df['Entry/Exit Position']).cumsum() 

In [101]:
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
LTC_signals_df['Portfolio Total'] = LTC_signals_df['Portfolio Cash'] + LTC_signals_df['Portfolio Holdings']

# Conclusion:  each point at a time, you can see what you have is 
# 1) if you are holding stock. say 500 share, what's holding value of 500 share at that moment + (initial capital + net of cost and proceed)
# in each row, you have  holding = position * close + initial investment - cost of previous buys and + proceed of previous sales

In [102]:
LTC_signals_df.loc["2021-02-02":"2022-10-20",:]

,close,SMA_fast,SMA_slow,Signal,Actual Returns,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total
timestamp,,,,,,,,,,,
2021-02-02 06:00:00+00:00,152.44,137.832000,134.5694,1.0,0.130777,0.0,50.0,0.0,7622.0,100000.0,107622.0
2021-02-03 06:00:00+00:00,151.33,137.842667,135.9950,1.0,-0.007282,0.0,50.0,0.0,7566.5,100000.0,107566.5
2021-02-04 06:00:00+00:00,148.29,138.216000,137.0894,1.0,-0.020089,0.0,50.0,0.0,7414.5,100000.0,107414.5
2021-02-05 06:00:00+00:00,158.76,140.150000,138.2412,1.0,0.070605,0.0,50.0,0.0,7938.0,100000.0,107938.0
2021-02-06 06:00:00+00:00,151.36,141.056667,139.1292,1.0,-0.046611,0.0,50.0,0.0,7568.0,100000.0,107568.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-10-16 05:00:00+00:00,51.43,52.754000,55.2348,0.0,0.014599,0.0,0.0,0.0,0.0,107604.0,107604.0
2022-10-17 05:00:00+00:00,51.86,52.738667,55.1828,0.0,0.008361,0.0,0.0,0.0,0.0,107604.0,107604.0
2022-10-18 05:00:00+00:00,51.57,52.566667,55.0948,0.0,-0.005592,0.0,0.0,0.0,0.0,107604.0,107604.0


In [103]:
#Train Test begin
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

In [104]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = LTC_signals_df[['SMA_fast', 'SMA_slow']].shift().dropna()

# Review the DataFrame
X

,SMA_fast,SMA_slow
timestamp,,
2020-12-17 06:00:00+00:00,80.876000,72.3172
2020-12-18 06:00:00+00:00,81.892667,73.2238
2020-12-19 06:00:00+00:00,83.083333,74.2796
2020-12-20 06:00:00+00:00,85.801333,75.6000
2020-12-21 06:00:00+00:00,87.890000,76.7890
...,...,...
2022-10-23 05:00:00+00:00,51.906667,54.6486
2022-10-24 05:00:00+00:00,51.932000,54.5188
2022-10-25 05:00:00+00:00,51.825333,54.3658


In [105]:
# Create the target set selecting the Signal column and assiging it to y
y = LTC_signals_df['Signal']

# Review the value counts
y.value_counts()

0.0    369
1.0    312
Name: Signal, dtype: int64

In [106]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

2020-12-17 06:00:00+00:00


In [107]:
# Select the ending period for the training data with an offset of 1 months
training_end = X.index.min() + DateOffset(months=16)

# Display the training end date
print(training_end)

2022-04-17 06:00:00+00:00


In [108]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
display(X_train.head())
display(X_train.tail())

,SMA_fast,SMA_slow
timestamp,,
2020-12-17 06:00:00+00:00,80.876000,72.3172
2020-12-18 06:00:00+00:00,81.892667,73.2238
2020-12-19 06:00:00+00:00,83.083333,74.2796
2020-12-20 06:00:00+00:00,85.801333,75.6000
2020-12-21 06:00:00+00:00,87.890000,76.7890


,SMA_fast,SMA_slow
timestamp,,
2022-04-13 05:00:00+00:00,118.090667,112.8754
2022-04-14 05:00:00+00:00,116.963333,112.9460
2022-04-15 05:00:00+00:00,115.672667,113.2542
2022-04-16 05:00:00+00:00,115.024667,113.4238
2022-04-17 05:00:00+00:00,114.182000,113.4932


In [109]:
display(y_train.head())
display(y_train.tail())
y_train.describe()

timestamp
2020-12-17 06:00:00+00:00    1.0
2020-12-18 06:00:00+00:00    1.0
2020-12-19 06:00:00+00:00    1.0
2020-12-20 06:00:00+00:00    1.0
2020-12-21 06:00:00+00:00    1.0
Name: Signal, dtype: float64

timestamp
2022-04-13 05:00:00+00:00    1.0
2022-04-14 05:00:00+00:00    1.0
2022-04-15 05:00:00+00:00    1.0
2022-04-16 05:00:00+00:00    1.0
2022-04-17 05:00:00+00:00    0.0
Name: Signal, dtype: float64

count    487.000000
mean       0.552361
std        0.497762
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: Signal, dtype: float64

In [110]:
X_train.describe()

,SMA_fast,SMA_slow
count,487.000000,487.000000
mean,168.285624,166.804960
std,48.408644,44.267415
min,80.876000,72.317200
25%,132.896000,134.064300
50%,160.561333,167.391800
75%,192.993667,196.151600
max,332.439333,271.604600


In [111]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
display(X_test.head())
display(X_test.tail())

,SMA_fast,SMA_slow
timestamp,,
2022-04-18 05:00:00+00:00,112.866000,113.4882
2022-04-19 05:00:00+00:00,111.931333,113.6638
2022-04-20 05:00:00+00:00,111.076000,113.6686
2022-04-21 05:00:00+00:00,110.534000,113.6478
2022-04-22 05:00:00+00:00,110.214667,113.5794


,SMA_fast,SMA_slow
timestamp,,
2022-10-23 05:00:00+00:00,51.906667,54.6486
2022-10-24 05:00:00+00:00,51.932000,54.5188
2022-10-25 05:00:00+00:00,51.825333,54.3658
2022-10-26 05:00:00+00:00,52.128667,54.2890
2022-10-27 05:00:00+00:00,52.396000,54.3394


In [112]:
# Scale the features DataFrames

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [113]:
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()

# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred[:10]
        

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [114]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred)

# Print the classification report
print(svm_testing_report)

              precision    recall  f1-score   support

         0.0       1.00      0.32      0.49       151
         1.0       0.29      1.00      0.45        42

    accuracy                           0.47       193
   macro avg       0.65      0.66      0.47       193
weighted avg       0.85      0.47      0.48       193



In [115]:
# Create a new empty predictions DataFrame:

# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = LTC_signals_df["Actual Returns"]

# Add the strategy returns to the DataFrame
predictions_df['Strategy Returns'] = LTC_signals_df["Actual Returns"]*predictions_df["Predicted"]

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

,Predicted,Actual Returns,Strategy Returns
timestamp,,,
2022-04-18 05:00:00+00:00,0.0,0.058257,0.0
2022-04-19 05:00:00+00:00,0.0,0.004550,0.0
2022-04-20 05:00:00+00:00,0.0,-0.005418,-0.0
2022-04-21 05:00:00+00:00,0.0,-0.039293,-0.0
2022-04-22 05:00:00+00:00,0.0,-0.008366,-0.0


,Predicted,Actual Returns,Strategy Returns
timestamp,,,
2022-10-23 05:00:00+00:00,1.0,0.024713,0.024713
2022-10-24 05:00:00+00:00,1.0,-0.013274,-0.013274
2022-10-25 05:00:00+00:00,1.0,0.065934,0.065934
2022-10-26 05:00:00+00:00,1.0,0.001600,0.001600
2022-10-27 05:00:00+00:00,1.0,-0.025555,-0.025555


In [116]:
# Plot the actual returns versus the strategy returns
(1 + predictions_df[["Actual Returns", "Strategy Returns"]]).cumprod().hvplot(width=1000,height=500)

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)